# CSCI 544 - Homework 2 - HMMs on part-of- speech tagging

In [73]:
import pandas as pd
import numpy as np

In [74]:
#reading the dataframe and renaming columns

df = pd.read_csv('data/train', sep='\t', on_bad_lines = 'skip')
df.columns = ['index', 'word', 'postag']
df.loc[-1] = [1, 'Pierre', 'NNP']  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 

In [75]:
df

,index,word,postag
0,1,Pierre,NNP
1,2,Vinken,NNP
2,3,",",","
3,4,61,CD
4,5,years,NNS
...,...,...,...
912090,22,to,TO
912091,23,San,NNP
912092,24,Francisco,NNP
912093,25,instead,RB


In [76]:
# setting threshold as 3

counts = df['word'].value_counts()
idx = counts[counts.lt(3)].index

df.loc[df['word'].isin(idx), 'word'] = '<unk>' 
df2 = df

In [77]:
df2 = df

In [78]:
df = df.drop(columns=['index'])

In [79]:
# getting value counts of each unique word
df['occurrences'] = df.groupby(['word'])['postag'].transform('count')

In [80]:
df = df.sort_values('occurrences', ascending = False)
df = df.drop(columns=['postag'])

In [81]:
df = df.drop_duplicates(keep='first')

In [82]:
df = df.reset_index()
df['index'] = df.index

In [83]:
df = df[['word', 'index', 'occurrences']]

In [84]:
df

,word,index,occurrences
0,",",0,46476
1,the,1,39533
2,.,2,37452
3,<unk>,3,32537
4,of,4,22104
...,...,...,...
16915,Carver,16915,3
16916,Minna,16916,3
16917,lien,16917,3
16918,Dooling,16918,3


In [85]:
vocab_list = df['word'].to_list()

In [86]:
# shifting results of unk on top

df=df.drop(3)
df.loc[-1] = ['<unk>',3, 32537]  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 

In [87]:
df.to_csv("vocab.txt", sep='\t', index=False)

In [88]:
df

,word,index,occurrences
0,<unk>,3,32537
1,",",0,46476
2,the,1,39533
3,.,2,37452
5,of,4,22104
...,...,...,...
16916,Carver,16915,3
16917,Minna,16916,3
16918,lien,16917,3
16919,Dooling,16918,3


In [89]:
#saving the file in the desired format

df["modified"] = df.apply(lambda x: "\\t".join(map(str, x)), axis=1)

# Save the modified data to a .txt file
filename = "vocab.txt"
with open(filename, "w") as file:
    file.write("\n".join(df["modified"]))

#### What is the selected threshold for unknown words replacement? = 3
#### What is the total size of your vocabulary? = 16920 unique words
#### What is the total occurrences of the special token ‘< unk >’ after replacement? = 32537


## Task 2: HMM Model Learning 

In [90]:
df2

,index,word,postag
0,1,Pierre,NNP
1,2,<unk>,NNP
2,3,",",","
3,4,61,CD
4,5,years,NNS
...,...,...,...
912090,22,to,TO
912091,23,San,NNP
912092,24,Francisco,NNP
912093,25,instead,RB


In [91]:
df2_dict = dict(zip(df2.word, df2.postag))

In [92]:
# list of unique postags and words

words = list((df2["word"]).unique())
postags = list((df2["postag"]).unique())

### 2.1 Creating Transition Probability dictionary

In [93]:
# create combinations including repeats

def postag_combos(list1, list2):
    combinations = []
    for i in list1:
        for j in list2:
            combinations.append((i, j))
    return combinations

postag_postag_combo = postag_combos(postags, postags)

In [94]:
#create a list of tuples 
postag_dict = {key: 0 for key in postag_postag_combo}
postag_list = list(postag_dict)

In [95]:
for i in range(1, len(df2)):
    pair = (df2.iloc[i-1]['postag'], df2.iloc[i]['postag'])
    if pair in postag_dict:
        postag_dict[pair] += 1
    

In [96]:
#calculating transition probab and storing it in a dictionary

transition_probab = {}
for i in range(0, len(postag_list)):
    if postag_list[i] in postag_dict:  
        transition_probab[postag_list[i]] = postag_dict[postag_list[i]] / (df2['postag'] == postag_list[i][0]).sum()
        

In [97]:
df2['postag'].nunique()

45

In [98]:
sum(transition_probab.values())

44.99997360293533

### 2.2 Creating Emission Probability dictionary

In [99]:
# create unique combinations of words and tags

postag_word_combo = [(x, y) for x in postags for y in words]
postag_word_dict = {key: 0 for key in postag_word_combo}

In [100]:
# creating a dictionary of tuples and value counts

for i in range(0, len(df2)):
    pair = (df2.iloc[i]['postag'], df2.iloc[i]['word'])
    if pair in postag_word_dict:
        postag_word_dict[pair] += 1

In [101]:
#value count of every POS tag in the dataset
postag_value_counts = df2['postag'].value_counts()

# convert value counts to a dictionary
value_counts_dict = postag_value_counts.to_dict()

In [102]:
#calculating emission probab and storing it in a dictionary

emission_probab = {}
for postag1 in postag_word_dict:
    emission_probab[postag1] = postag_word_dict[postag1] / value_counts_dict[postag1[0]]

In [103]:
sum(emission_probab.values())

45.00000000000072

In [104]:
#saving the values in a json

import json 


tp = {str(k): v for k, v in transition_probab.items()}
ep = {str(k): v for k, v in emission_probab.items()}

with open('hmm.json', 'w') as f:
    json.dump(tp, f)
    json.dump(ep, f)

# Task 3: Greedy Decoding Model

### Generate initial probabilities with formula: 
    - total value counts of each postag / the total length of the dataset


In [105]:
#Create initial probabilities

initial_probability = df2['postag'].value_counts(ascending=False) / len(df2)
initial_probability = dict(initial_probability) #converting to dictionary
initial_probability = {k: v for k, v in sorted(initial_probability.items(), key=lambda item: list(df2['postag'].unique()).index(item[0]))}
initial_probability = list(initial_probability.values())

In [106]:
# reading dev data

df_gd = pd.read_csv('data/dev', sep='\t', on_bad_lines = 'skip')
df_gd.columns = ['index', 'word', 'postag']
df_gd.loc[-1] = ['1', 'The', 'DT']  # adding a row
df_gd.index = df_gd.index + 1  # shifting index
df_gd.sort_index(inplace=True)

In [107]:
df_gd

,index,word,postag
0,1,The,DT
1,2,Arizona,NNP
2,3,Corporations,NNP
3,4,Commission,NNP
4,5,authorized,VBD
...,...,...,...
131763,13,join,VB
131764,14,the,DT
131765,15,winning,VBG
131766,16,bidder,NN


In [108]:
len_postags = len(postags)
len_words = len(words)

### 3.1 Transition Probability Dataframe

In [109]:
# converting Transition Probability dictionary to matrix 

trans_prob_matrix = [[0 for j in range(len(postags))] for i in range(len(postags))]

for i in range(len(postags)):
    for j in range(len(postags)):
        trans_prob_matrix[i][j] = transition_probab.get((postags[i], postags[j]), 0)

In [110]:
#converting Transition Probability matrix to dataframe

transition_df = pd.DataFrame(trans_prob_matrix, columns = list(postags), index=list(postags))

In [111]:
transition_df

,NNP,",",CD,NNS,JJ,MD,VB,DT,NN,IN,...,WP$,-LRB-,-RRB-,PDT,RBS,FW,UH,SYM,LS,#
NNP,0.378870,0.138469,0.019176,0.024438,0.008549,0.011163,0.001050,0.002545,0.057666,0.041046,...,0.000000,0.003299,0.003630,0.000011,0.000011,0.000502,0.000000,0.000034,0.000000,0.000000
",",0.126549,0.000000,0.021235,0.027345,0.040964,0.010542,0.003894,0.133670,0.049118,0.085650,...,0.002044,0.000301,0.000000,0.000215,0.000366,0.000301,0.000366,0.000000,0.000022,0.000000
CD,0.012932,0.095481,0.199564,0.157759,0.037160,0.002064,0.000057,0.028157,0.203320,0.089288,...,0.000029,0.001692,0.008860,0.000000,0.000201,0.000057,0.000000,0.000143,0.000000,0.000000
NNS,0.003146,0.123905,0.001711,0.010681,0.017439,0.027965,0.003941,0.016920,0.020481,0.234553,...,0.000501,0.004390,0.001659,0.000000,0.000138,0.000000,0.000000,0.000052,0.000000,0.000000
JJ,0.036933,0.029129,0.016236,0.233052,0.074002,0.000407,0.000119,0.003631,0.449104,0.056528,...,0.000000,0.000611,0.000475,0.000017,0.000102,0.000136,0.000017,0.000000,0.000000,0.000085
MD,0.000636,0.003497,0.000212,0.000530,0.000318,0.000106,0.799089,0.004027,0.000530,0.001589,...,0.000000,0.000212,0.000212,0.000000,0.000212,0.000000,0.000000,0.000000,0.000000,0.000000
VB,0.031935,0.017223,0.020440,0.051316,0.086547,0.000510,0.005296,0.222135,0.062105,0.110322,...,0.000078,0.001020,0.000353,0.001569,0.000314,0.000078,0.000118,0.000000,0.000000,0.000118
DT,0.111203,0.002145,0.022901,0.073983,0.218343,0.002171,0.000203,0.001714,0.473488,0.009889,...,0.000000,0.000508,0.000038,0.000000,0.002805,0.000279,0.000013,0.000000,0.000000,0.000140
NN,0.009590,0.113578,0.006140,0.078685,0.008978,0.017721,0.001388,0.006869,0.122187,0.247448,...,0.000180,0.001654,0.001764,0.000008,0.000086,0.000063,0.000000,0.000055,0.000008,0.000016
IN,0.148747,0.002522,0.059320,0.059879,0.091127,0.000116,0.000454,0.328120,0.109363,0.020136,...,0.000032,0.000327,0.000074,0.001425,0.000179,0.000190,0.000011,0.000021,0.000011,0.000559


### 3.2 Emission Probability Dataframe

In [112]:
#converting Emission Probability dictionary to matrix 

emission_probab_matrix = [[0 for j in range(len(words))] for i in range(len(postags))]

for i in range(len(postags)):
    for j in range(len(words)):
        emission_probab_matrix[i][j] = emission_probab.get((postags[i],words[j]), 0)

In [113]:
#converting Emission Probability matrix to dataframe 

emission_df = pd.DataFrame(emission_probab_matrix, columns = list(words), index=list(postags))

In [114]:
emission_df

,Pierre,<unk>,",",61,years,old,will,join,the,board,...,Ameritech,Libor,prince,CDL,Equitec,Steinhardt,Granges,Tartan,Sasea,Fiorini
NNP,0.000068,0.093074,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000068,0.000000,...,0.000034,0.000034,0.000000,0.000057,0.000068,0.000388,0.000023,0.000034,0.000034,0.000034
",",0.000000,0.000000,0.999914,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CD,0.000000,0.120312,0.000000,0.000717,0.00000,0.000000,0.000000,0.000000,0.000029,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NNS,0.000000,0.056033,0.000000,0.000000,0.01953,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000035,0.000000,0.000000,0.000000
JJ,0.000000,0.098042,0.000000,0.000000,0.00000,0.003614,0.000000,0.000000,0.000119,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MD,0.000000,0.000530,0.000000,0.000000,0.00000,0.000000,0.313871,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VB,0.000000,0.031504,0.000000,0.000000,0.00000,0.000000,0.000039,0.001569,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DT,0.000000,0.000051,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.501644,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NN,0.000000,0.041769,0.000000,0.000000,0.00000,0.000000,0.000149,0.000000,0.000008,0.002329,...,0.000000,0.000000,0.000024,0.000000,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000
IN,0.000000,0.000401,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [115]:
def calculate_nonzero_probs(transition_df, emission_df):
    transition_count = (transition_df != 0).sum().sum()
    emission_count = (emission_df != 0).sum().sum()
    return transition_count, emission_count

In [116]:
transition_count, emission_count = calculate_nonzero_probs(transition_df, emission_df)
print("Number of non-zero transition probabilities:", transition_count)
print("Number of non-zero emission probabilities:", emission_count)

Number of non-zero transition probabilities: 1378
Number of non-zero emission probabilities: 23373


### 3.3 Greedy Decoding - Dev

In [117]:
# Greedy Decoding - dev

def greedy_decoder(dataframe):
    greedy_predicted_list = []
    first_word = 1 #setting it as true

    for i in dataframe.index:
        dev_word = dataframe['word'][i] #iterate over every word in the dataframe
        if(dev_word not in words): #check if word exists in vocabulary
            dev_word = "<unk>"   

        if(dev_word == "."): # check if a full stop is encountered
            first_word = 1
            greedy_predicted_list.append(dev_word)

        elif(first_word == 1): #check if it is first word
            first_word = 0 #after first word, set flag to 0
            inital_x_emission = initial_probability * emission_df[dev_word] # multiplying initial and emission probabilities
            max_probab = np.argmax(inital_x_emission) #get max probability out of the list
            greedy_predicted_list.append(postags[max_probab])     

        else: # check if it word after first word 
            emission_col = emission_df[dev_word]
            transition_col = transition_df.iloc[max_probab] 
            emission_x_transition = emission_col * transition_col # multiplying transition and emission probabilities
            max_probab = np.argmax(emission_x_transition) # get max probability
            greedy_predicted_list.append(postags[max_probab])
            
    return greedy_predicted_list

In [118]:
dev_predicted_list = greedy_decoder(df_gd)

In [119]:
accurate_words = 0

for i in df_gd.index:
    if df_gd['postag'][i] == dev_predicted_list[i]:
        accurate_words += 1
        
accuracy = accurate_words / len(df_gd)     

print("Accuracy of Greedy Decoding model:", accuracy * 100, "%")

Accuracy of Greedy Decoding model: 93.04383461842025 %


### 3.4 Generating POS tags for test

In [185]:
df_test = pd.read_csv('data/test', sep='\t', on_bad_lines = 'skip')
df_test.columns = ['index', 'word']
df_test.loc[-1] = ['1', 'Influential']  # adding a row
df_test.index = df_test.index + 1  # shifting index
df_test.sort_index(inplace=True)

In [186]:
df_test

,index,word
0,1,Influential
1,2,members
2,3,of
3,4,the
4,5,House
...,...,...
129649,26,them
129650,27,here
129651,28,with
129652,29,us


In [187]:
greedy_test_predicted_list = greedy_decoder(df_test)

In [188]:
df_test['postag'] = greedy_test_predicted_list

In [189]:
df_test

,index,word,postag
0,1,Influential,NNP
1,2,members,NNS
2,3,of,IN
3,4,the,DT
4,5,House,NNP
...,...,...,...
129649,26,them,PRP
129650,27,here,RB
129651,28,with,IN
129652,29,us,PRP


In [190]:
df_test.to_csv('greedy.out', sep='\t', header=None, index=None)


In [191]:
with open('greedy.out', 'w') as f:
    
    f.write("1" + "\t" + df_test.iloc[0]['word'] + "\t" + str(greedy_test_predicted_list[0]) + "\n" )
    
    count = 2
    for i in range(1, len(df_test)):
        if(df_test.iloc[i]['index'] == 1):
            count = 1
            f.write("\n")
        f.write(str(count) + "\t" + df_test.iloc[i]['word'] + "\t" + str(greedy_test_predicted_list[i]) + "\n" )
        count+=1

## Task 4: Viterbi Decoding with HMM 

In [192]:
# read dev data

df_gd = pd.read_csv('data/dev', sep='\t', on_bad_lines = 'skip')
df_gd.columns = ['index', 'word', 'postag']
df_gd.loc[-1] = ['1', 'The', 'DT']  # adding a row
df_gd.index = df_gd.index + 1  # shifting index
df_gd.sort_index(inplace=True) 

### 4.1 Viterbi Decoding - Dev

In [193]:
# Viterbi Decoding

def viterbi_decoder(dataframe):
    
    final_node_list = []
    first_word = 1
    viterbi_predicted_list = []

    for i in dataframe.index: 
        dev_word = dataframe['word'][i] #iterate
        if(dev_word not in words): #check if word exists in vocabulary
            dev_word = "<unk>"   

        if(first_word == 1):
            first_word  = 0
            prev_probabilities = [-1] * len(initial_probability) #setting all probabilities to -1 for first word
            initial_x_emission = initial_probability * emission_df[dev_word] # multiplying initial and emission probabilities
            prev_probabilities = list(initial_x_emission)
            prev_nodes = [] #creating a list of the indexes of the POS tags with the highest probabilty for that iteration 
            for index, val in enumerate(prev_probabilities):
                if val > 0:
                    prev_nodes.append(index) 
            final_node_list= []

        elif(dev_word != '.'): # check if its the word after first word
            final_probabilities = [-1] * len(initial_probability) #setting all probabilities to -1 for first word
            final_nodes = [-1] * len(initial_probability) #setting all indexes to -1 for first word

            emission_probability = list(emission_df[dev_word])

            for k in prev_nodes: # iterate over the indexes of POS tags

                transition_list = list(transition_df.iloc[k])
                transition_probability = [(itr * prev_probabilities[k]) for itr in transition_list]
                curr_probability = [a*b for a,b in zip(transition_probability, emission_probability)]

                for m in range(len(initial_probability)):
                    if curr_probability[m] > final_probabilities[m]: # if the current probability is greater than the previous probability
                        final_probabilities[m] = curr_probability[m] # replace the specific probability with the new probability
                        final_nodes[m] = k # same for nodes or index of POS tag
            final_node_list.append(final_nodes)
            prev_probabilities = final_probabilities

            prev_nodes = []
            for index, val in enumerate(prev_probabilities):
                if val > 0:
                    prev_nodes.append(index)  
        else: # check if its a full stop
            first_word = 1
            final_probabilities = [-1] * len(initial_probability)
            final_nodes = [-1] * len(initial_probability)
            emission_probability = list(emission_df[dev_word])

            for k in prev_nodes: 
                transition_list = list(transition_df.iloc[:,k])
                transition_probability = [(itr * prev_probabilities[k]) for itr in transition_list]
                curr_probability = [a*b for a,b in zip(transition_probability, emission_probability)] # multiply item by item

                for m in range(len(initial_probability)):
                    if curr_probability[m] > final_probabilities[m]:
                        final_probabilities[m] = curr_probability[m] 
                        final_nodes[m] = k

            final_node_list.append(final_nodes)
            prev_probabilities = final_probabilities

            prev_nodes = []

            for index, val in enumerate(prev_probabilities):
                if val > 0:
                    prev_nodes.append(index) 

            tags_for_this_sentence = []
            first_word  = 1

            tag_for_last_word = np.argmax(prev_probabilities)
            tags_for_this_sentence.append(postags[tag_for_last_word])
            length_sentence = len(final_node_list) - 1

            while(length_sentence >= 0 ): # tracing back nodes to get the highest probability for the sentence
                current_postag = final_node_list[length_sentence][tag_for_last_word]
                length_sentence = length_sentence - 1
                tags_for_this_sentence.append(postags[current_postag])
                tag_for_last_word = current_postag
            tags_for_this_sentence.reverse()

            viterbi_predicted_list.append(tags_for_this_sentence)

    viterbi_predicted_list =  [item for sub_list in viterbi_predicted_list for item in sub_list]
    
    return viterbi_predicted_list

### 4.2 Testing on Dev dataset

In [194]:
#generate list of predicted tags

dev_viterbi_predicted_list = viterbi_decoder(df_gd)

In [195]:
accurate_words = 0

for i in range(len(dev_viterbi_predicted_list)):
    if df_gd['postag'][i] == dev_viterbi_predicted_list[i]:
        accurate_words += 1
        
accuracy = accurate_words / len(dev_viterbi_predicted_list)     

print("Accuracy of Viterbi model of Dev data:", accuracy * 100, "%")

Accuracy of Viterbi model of Dev data: 94.28465181227612 %


### 4.3 Generating results for test dataset

In [196]:
#read test dataset

df_test_2 = pd.read_csv('data/test', sep='\t', on_bad_lines = 'skip')
df_test_2.columns = ['index', 'word']
df_test_2.loc[-1] = ['1', 'Influential']  # adding a row
df_test_2.index = df_test_2.index + 1  # shifting index
df_test_2.sort_index(inplace=True)

In [197]:
#generate list of predicted tags

viterbi_test_predicted_list = viterbi_decoder(df_test_2)

In [198]:
df_test_2['postag'] = viterbi_test_predicted_list

In [199]:
df_test_2

,index,word,postag
0,1,Influential,JJ
1,2,members,NNS
2,3,of,IN
3,4,the,DT
4,5,House,NNP
...,...,...,...
129649,26,them,PRP
129650,27,here,RB
129651,28,with,IN
129652,29,us,PRP


In [200]:
df_test_2.to_csv('viterbi.out', sep='\t', header=None, index=None)


In [201]:
with open('viterbi.out', 'w') as f:
    
    f.write("1" + "\t" + df_test_2.iloc[0]['word'] + "\t" + str(viterbi_test_predicted_list[0]) + "\n" )
    
    count = 2
    for i in range(1, len(df_test)):
        if(df_test_2.iloc[i]['index'] == 1):
            count = 1
            f.write("\n")
        f.write(str(count) + "\t" + df_test_2.iloc[i]['word'] + "\t" + str(viterbi_test_predicted_list[i]) + "\n" )
        count+=1

In [202]:
!jupyter nbconvert --to script HW2.ipynb


[NbConvertApp] Converting notebook HW2.ipynb to script
[NbConvertApp] Writing 15742 bytes to HW2.py
